# Learning to Rank

## Feature extraction

### Training data

In [1]:
import pandas as pd
from pyserini.index import IndexReader

passages = pd.read_csv('collections/msmarco-passage/collectionandqueries/collection.tsv', sep = '\t', names=['pid', 'passage'])
queries_train = pd.read_csv('collections/msmarco-passage/collectionandqueries/queries.train.tsv', sep = '\t', names=['qid', 'query'])
qrels_train = pd.read_csv('collections/msmarco-passage/collectionandqueries/qrels.train.tsv', sep = '\t', names=['qid', 'Q0', 'docid', 'rating'])

print(passages)
print(queries_train)
print(qrels_train)

             pid                                            passage
0              0  The presence of communication amid scientific ...
1              1  The Manhattan Project and its atomic bomb help...
2              2  Essay on The Manhattan Project - The Manhattan...
3              3  The Manhattan Project was the name for a proje...
4              4  versions of each volume as well as complementa...
...          ...                                                ...
8841818  8841818  When metal salts emit short wavelengths of vis...
8841819  8841819  Thousands of people across the United States w...
8841820  8841820  The recipe that creates blue, for example, inc...
8841821  8841821  On Independence Days of yore, old-timey crowds...
8841822  8841822  View full size image. Behind the scenes of the...

[8841823 rows x 2 columns]
            qid                                           query
0        121352                                  define extreme
1        634306        what 

In [2]:
import numpy as np
from tqdm import tqdm

#index_reader = IndexReader('indexes/collection_jsonl')
index_reader = IndexReader.from_prebuilt_index('msmarco-passage')

training_data = []

for index in tqdm(range(len(qrels_train))):
    row = qrels_train.loc[index]
    query = queries_train[queries_train['qid'] == row['qid']]['query'].values[0]
    passage = passages[passages['pid'] == row['docid']]['passage'].values[0]
    rating = row['rating']
    
    bm25_score = index_reader.compute_query_document_score(str(row['docid']), query)
    
    passage_length = len(passage)

    c = 0
    df = 0
    cf = 0
    idf = 0
    c_idf = 0
    C = len(passages)
    passage_analyzed = index_reader.analyze(passage)
    for term in index_reader.analyze(query):
        if index_reader.analyze(term) == []:
            continue
        else:
            c += passage_analyzed.count(term)
            df_temp, cf_temp = index_reader.get_term_counts(term) 
            df += df_temp
            cf += cf_temp
            idf += np.log((C - df + 0.5) / (df + 0.5))

            c_idf += c * idf
    
    temp = [row['qid'], row['docid'], row['rating'], bm25_score, passage_length, c, df, cf, idf, c_idf]
    training_data.append(temp)

Attempting to initialize pre-built index msmarco-passage.
/Users/jessie/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d already exists, skipping download.
Initializing msmarco-passage...


  3%|▎         | 18292/532761 [05:06<2:05:07, 68.53it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
  5%|▍         | 25651/532761 [07:02<2:01:02, 69.83it/s] <ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
  6%|▌         | 31139/532761 [08:47<2:47:54, 49.79it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
  7%|▋         | 37889/532761 [10:31<2:23:15, 57.57it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
  8%|▊         | 44509/532761 [12:16<1:53:51, 71.47it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
 10%|█         | 53993/532761 [14:44<2:10:44, 61.03it/s]<ipython-input-2-c

 91%|█████████ | 483775/532761 [2:01:26<09:57, 82.04it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
 93%|█████████▎| 495455/532761 [2:04:11<08:34, 72.50it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
 93%|█████████▎| 496517/532761 [2:04:25<08:04, 74.75it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
 96%|█████████▋| 512977/532761 [2:07:58<04:14, 77.60it/s]<ipython-input-2-c870f1d99fb6>:34: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
100%|██████████| 532761/532761 [2:12:38<00:00, 66.94it/s]


In [19]:
training_data = pd.DataFrame(training_data, columns=['qid', 'docid', 'rating', 'bm25', \
                                                     'passage_length', 'c', 'df', 'cf', \
                                                     'idf', 'c_idf']).dropna()
training_data.to_csv('training_data.csv')
print(training_data)

            qid    docid  rating       bm25  passage_length   c       df  \
0       1185869        0       1  11.560829             325   5   797858   
1       1185868       16       1  20.567997             306   9   917868   
2        597651       49       1  11.347543             305   6   718082   
3        403613       60       1  13.854973             521   7   200763   
4       1183785      389       1  10.219151             319   3   390237   
...         ...      ...     ...        ...             ...  ..      ...   
532756    19285  8841362       1   8.897675             304   2   209748   
532757   558837  4989159       1   3.825954             297   3   612669   
532758   559149  8841547       1  13.113671             214   4  1068484   
532759   706678  8841643       1   7.690524             149   1   543276   
532760   405466  8841735       1  10.627259             798  13   155130   

             cf        idf        c_idf  
0        983404  15.112933   173.856118  
1  

In [4]:
import random

training_data_nonrelevant = []

for index in tqdm(random.sample(range(len(qrels_train)), 10000)):
    row = qrels_train.loc[index]
    query = queries_train[queries_train['qid'] == row['qid']]['query'].values[0]
    docid = passages[passages['pid'] != row['docid']]['pid'].sample().values[0]
    passage = passages[passages['pid'] == docid]['passage'].values[0]
    rating = row['rating']
    
    bm25_score = index_reader.compute_query_document_score(str(docid), query)
    
    passage_length = len(passage)

    c = 0
    df = 0
    cf = 0
    idf = 0
    c_idf = 0
    C = len(passages)
    passage_analyzed = index_reader.analyze(passage)
    for term in index_reader.analyze(query):
        if index_reader.analyze(term) == []:
            continue
        else:
            c += passage_analyzed.count(term)
            df_temp, cf_temp = index_reader.get_term_counts(term) 
            df += df_temp
            cf += cf_temp
            idf += np.log((C - df + 0.5) / (df + 0.5))

            c_idf += c * idf
    
    temp = [row['qid'], docid, 0, bm25_score, passage_length, c, df, cf, idf, c_idf]
    training_data_nonrelevant.append(temp)

100%|██████████| 10000/10000 [5:02:04<00:00,  1.81s/it]  


In [20]:
training_data_nonrelevant = pd.DataFrame(training_data_nonrelevant, columns=['qid', 'docid', 'rating', 'bm25', \
                                                                             'passage_length', 'c', 'df', 'cf', \
                                                                             'idf', 'c_idf']).dropna()
training_data_nonrelevant.to_csv('training_data_nonrelevant.csv')
print(training_data_nonrelevant)

          qid    docid  rating      bm25  passage_length  c       df       cf  \
0      539478  2693464       0  0.000000             258  0    91835   113542   
1      400705  7629915       0  0.000000             205  0   150809   186927   
2        2797  3365674       0  0.000000             282  0   622866   810154   
3      734390  6235369       0  0.000000             505  0   780361   969702   
4      345809   971793       0  0.000000             296  0   756389   955734   
...       ...      ...     ...       ...             ... ..      ...      ...   
9995    92552  5884425       0  0.000000             268  0   221142   264779   
9996   435696  2716350       0  2.086764             304  1   221593   267426   
9997  1181703  8602111       0  0.000000             321  0   486729   732203   
9998  1173262  3786944       0  0.000000             737  0   122353   181461   
9999   650309  7430049       0  0.000000             334  0  1645114  2377775   

            idf      c_idf 

In [43]:
with open('training_data.txt', 'w') as file:
    sample = training_data.sample(10000)
    for index in tqdm(range(len(sample))):
        row = sample.iloc[index]
        file.write('{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} # docid = {} \n'.format(int(row['rating']), int(row['qid']), \
                                                                         row['bm25'], row['passage_length'], \
                                                                        row['c'], row['df'], row['cf'], \
                                                                        row['idf'], row['c_idf'], int(row['docid'])))
    for index in tqdm(range(len(training_data_nonrelevant))):
        row = training_data_nonrelevant.iloc[index]
        file.write('{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} # docid = {} \n'.format(int(row['rating']), int(row['qid']), \
                                                                        row['bm25'], row['passage_length'], \
                                                                        row['c'], row['df'], row['cf'], \
                                                                        row['idf'], row['c_idf'], int(row['docid'])))

100%|██████████| 10000/10000 [00:03<00:00, 2747.59it/s]


### Validation data

In [8]:
queries_val = pd.read_csv('collections/msmarco-passage/collectionandqueries/queries.dev.small.tsv', sep = '\t', names=['qid', 'query'])
qrels_val = pd.read_csv('collections/msmarco-passage/collectionandqueries/qrels.dev.small.tsv', sep = '\t', names=['qid', 'Q0', 'docid', 'rating'])

print(queries_val)
print(qrels_val)

          qid                                          query
0     1048585                   what is paula deen's brother
1           2                       Androgen receptor define
2      524332  treating tension headaches without medication
3     1048642                            what is paranoid sc
4      524447            treatment of varicose veins in legs
...       ...                                            ...
6975   734979                           what is coronary cta
6976   524166              transportation to us bank stadium
6977   968921                 where did last names originate
6978   786375                 what is preoperative clearance
6979  1048565                  who plays sebastian michaelis

[6980 rows x 2 columns]
         qid  Q0    docid  rating
0     300674   0  7067032       1
1     125705   0  7067056       1
2      94798   0  7067181       1
3       9083   0  7067274       1
4     174249   0  7067348       1
...      ...  ..      ...     ...
7432 

In [9]:
validation_data = []

for index in tqdm(range(len(qrels_val))):
    row = qrels_val.loc[index]
    query = queries_val[queries_val['qid'] == row['qid']]['query'].values[0]
    passage = passages[passages['pid'] == row['docid']]['passage'].values[0]
    rating = row['rating']
    
    bm25_score = index_reader.compute_query_document_score(str(row['docid']), query)
    
    passage_length = len(passage)

    c = 0
    df = 0
    cf = 0
    idf = 0
    c_idf = 0
    C = len(passages)
    passage_analyzed = index_reader.analyze(passage)
    for term in index_reader.analyze(query):
        if index_reader.analyze(term) == []:
            continue
        else:
            c += passage_analyzed.count(term)
            df_temp, cf_temp = index_reader.get_term_counts(term) 
            df += df_temp
            cf += cf_temp
            idf += np.log((C - df + 0.5) / (df + 0.5))

            c_idf += c * idf
    
    temp = [row['qid'], row['docid'], row['rating'], bm25_score, passage_length, c, df, cf, idf, c_idf]
    validation_data.append(temp)

 29%|██▊       | 2132/7437 [00:31<01:30, 58.77it/s]<ipython-input-9-867129774ef1>:28: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
 53%|█████▎    | 3928/7437 [00:55<00:46, 75.94it/s]<ipython-input-9-867129774ef1>:28: RuntimeWarning: invalid value encountered in log
  idf += np.log((C - df + 0.5) / (df + 0.5))
100%|██████████| 7437/7437 [01:46<00:00, 69.57it/s]


In [23]:
validation_data = pd.DataFrame(validation_data, columns=['qid', 'docid', 'rating', 'bm25', \
                                                         'passage_length', 'c', 'df', 'cf', \
                                                         'idf', 'c_idf']).dropna()
validation_data.to_csv('validation_data.csv')
print(validation_data)

         qid    docid  rating       bm25  passage_length   c       df  \
0     300674  7067032       1  24.918810             324   9  2081477   
1     125705  7067056       1   3.744147             186   4   211104   
2      94798  7067181       1  13.672122             542   9   135438   
3       9083  7067274       1  11.293467             169   4   329167   
4     174249  7067348       1   8.314486             630  16  2118995   
...      ...      ...     ...        ...             ...  ..      ...   
7432  147073  8008770       1  11.880719             257   8  1126133   
7433  243761  8008787       1  12.142422             497   7  1083447   
7434  162662  8008977       1  17.052584             220   4  2425838   
7435  247194  8009319       1   8.106571             262   3  3267541   
7436  195199  8009377       1   5.757708             233   1   390524   

           cf        idf       c_idf  
0     2715181  14.770192  395.896149  
1      248706   8.136258   32.545033  
2     

In [49]:
validation_data_nonrelevant = []

for index in tqdm(random.sample(range(len(qrels_val)), 1000)):
    row = qrels_val.loc[index]
    query = queries_val[queries_val['qid'] == row['qid']]['query'].values[0]
    docid = passages[passages['pid'] != row['docid']]['pid'].sample().values[0]
    passage = passages[passages['pid'] == docid]['passage'].values[0]
    rating = row['rating']
    
    bm25_score = index_reader.compute_query_document_score(str(docid), query)
    
    passage_length = len(passage)

    c = 0
    df = 0
    cf = 0
    idf = 0
    c_idf = 0
    C = len(passages)
    passage_analyzed = index_reader.analyze(passage)
    for term in index_reader.analyze(query):
        if index_reader.analyze(term) == []:
            continue
        else:
            c += passage_analyzed.count(term)
            df_temp, cf_temp = index_reader.get_term_counts(term) 
            df += df_temp
            cf += cf_temp
            idf += np.log((C - df + 0.5) / (df + 0.5))

            c_idf += c * idf
    
    temp = [row['qid'], docid, 0, bm25_score, passage_length, c, df, cf, idf, c_idf]
    validation_data_nonrelevant.append(temp)

  0%|          | 4/1000 [00:16<1:06:40,  4.02s/it]


KeyboardInterrupt: 

In [54]:
validation_data_nonrelevant = pd.DataFrame(validation_data_nonrelevant, columns=['qid', 'docid', 'rating', \
                                                                                 'bm25', 'passage_length', \
                                                                                 'c', 'df', 'cf', 'idf', \
                                                                                 'c_idf']).dropna()
validation_data_nonrelevant.to_csv('validation_data_nonrelevant.csv')
print(validation_data_nonrelevant)

         qid    docid  rating  bm25  passage_length  c       df       cf  \
0      27743  4682722       0   0.0             600  0   529555   788281   
1     518675  1030433       0   0.0             242  0   350389   501527   
2     727765  3245041       0   0.0             290  0   861528  1071565   
3    1096619  7465244       0   0.0             293  0  1075196  1341418   
4    1095928  7857386       0   0.0             235  0   953099  1231183   
..       ...      ...     ...   ...             ... ..      ...      ...   
994   988954  3187724       0   0.0             261  0   547616   674387   
995   402427  6425929       0   0.0             265  0   224504   288905   
996  1046475   169071       0   0.0             102  0   554641   679486   
997   789997  6375925       0   0.0             259  0  1320154  1852520   
998  1080968  6683466       0   0.0             259  0   528991   777616   

           idf  c_idf  
0    19.183001    0.0  
1    20.578482    0.0  
2    10.212658 

In [55]:
with open('validation_data.txt', 'w') as file:
    sample = validation_data.sample(1000)
    for index in tqdm(range(len(sample))):
        row = sample.iloc[index]
        file.write('{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} # docid = {} \n'.format(int(row['rating']), int(row['qid']), \
                                                                         row['bm25'], row['passage_length'], \
                                                                        row['c'], row['df'], row['cf'], \
                                                                        row['idf'], row['c_idf'], int(row['docid'])))
    for index in tqdm(range(len(validation_data_nonrelevant))):
        row = validation_data_nonrelevant.iloc[index]
        file.write('{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} # docid = {} \n'.format(int(row['rating']), int(row['qid']), \
                                                                        row['bm25'], row['passage_length'], \
                                                                        row['c'], row['df'], row['cf'], \
                                                                        row['idf'], row['c_idf'], int(row['docid'])))

100%|██████████| 999/999 [00:00<00:00, 1213.30it/s]


### Testing data

In [15]:
queries_test = pd.read_csv('collections/msmarco-passage/msmarco-test2019-queries.tsv', sep = '\t', names=['qid', 'query'])
qrels_test = pd.read_csv('collections/msmarco-passage/2019qrels-pass.txt', sep = ' ', names=['qid', 'Q0', 'docid', 'rating'])

print(queries_test)
print(qrels_test)

         qid                                              query
0    1108939                  what slows down the flow of blood
1    1112389            what is the county for grand rapids, mn
2     792752                                     what is ruclip
3    1119729  what do you do when you have a nosebleed from ...
4    1105095                  where is sugar lake lodge located
..       ...                                                ...
195   146187  difference between a mcdouble and a double che...
196   634428                           what does chs stand for?
197  1121986     what are the effects of having low blood sugar
198   321441                 how much is a us postal stamp cost
199   532603                   university of dubuque enrollment

[200 rows x 2 columns]
          qid  Q0    docid  rating
0       19335  Q0  1017759       0
1       19335  Q0  1082489       0
2       19335  Q0   109063       0
3       19335  Q0  1160863       0
4       19335  Q0  1160871       

In [16]:
testing_data = []

for index in tqdm(range(len(qrels_test))):
    row = qrels_test.loc[index]
    query = queries_test[queries_test['qid'] == row['qid']]['query'].values[0]
    passage = passages[passages['pid'] == row['docid']]['passage'].values[0]
    rating = row['rating']
    
    bm25_score = index_reader.compute_query_document_score(str(row['docid']), query)
    
    passage_length = len(passage)

    c = 0
    df = 0
    cf = 0
    idf = 0
    c_idf = 0
    C = len(passages)
    passage_analyzed = index_reader.analyze(passage)
    for term in index_reader.analyze(query):
        if index_reader.analyze(term) == []:
            continue
        else:
            c += passage_analyzed.count(term)
            df_temp, cf_temp = index_reader.get_term_counts(term) 
            df += df_temp
            cf += cf_temp
            idf += np.log((C - df + 0.5) / (df + 0.5))

            c_idf += c * idf
    
    temp = [row['qid'], row['docid'], row['rating'], bm25_score, passage_length, c, df, cf, idf, c_idf]
    testing_data.append(temp)

100%|██████████| 9260/9260 [02:25<00:00, 63.50it/s]


In [27]:
testing_data = pd.DataFrame(testing_data, columns=['qid', 'docid', 'rating', 'bm25', 'passage_length', \
                                                   'c', 'df', 'cf', 'idf', 'c_idf'])
testing_data.to_csv('testing_data.csv')
print(testing_data)

          qid    docid  rating      bm25  passage_length  c      df      cf  \
0       19335  1017759       0  3.901871             309  3  270291  322485   
1       19335  1082489       0  6.753509             494  2  270291  322485   
2       19335   109063       0  6.547346             351  3  270291  322485   
3       19335  1160863       0  4.193603             279  4  270291  322485   
4       19335  1160871       0  4.209503             273  4  270291  322485   
...       ...      ...     ...       ...             ... ..     ...     ...   
9255  1133167  8839920       2  9.100336             252  5  649104  869667   
9256  1133167  8839922       2  7.480691             250  3  649104  869667   
9257  1133167   944810       0  2.127326             586  1  649104  869667   
9258  1133167   949411       0  2.163620             577  1  649104  869667   
9259  1133167   977421       0  6.752583             271  6  649104  869667   

            idf       c_idf  
0     15.600364   46.

In [28]:
with open('testing_data.txt', 'w') as file:
    for index in tqdm(range(len(testing_data))):
        row = testing_data.iloc[index]
        file.write('{} qid:{} 1:{} 2:{} 3:{} 4:{} 5:{} 6:{} 7:{} # docid = {} \n'.format(int(row['rating']), int(row['qid']), \
                                                                         row['bm25'], row['passage_length'], \
                                                                        row['c'], row['df'], row['cf'], \
                                                                        row['idf'], row['c_idf'], int(row['docid'])))

100%|██████████| 9260/9260 [00:02<00:00, 3601.77it/s]


## Model creation

```java -jar RankLib-2.15.jar -train training_data.txt -test testing_data.txt -validate validation_data.txt -ranker 6 -metric2t MAP -save LambdaMART.txt```

Result:
```
Discard orig. features
Training data:	training_data.txt
Test data:	testing_data.txt
Validation data:	validation_data.txt
Feature vector representation: Dense.
Ranking method:	LambdaMART
Feature description file:	Unspecified. All features will be used.
Train metric:	MAP
Test metric:	MAP
Feature normalization: No
Model file: LambdaMART.txt

[+] LambdaMART's Parameters:
No. of trees: 1000
No. of leaves: 10
No. of threshold candidates: 256
Min leaf support: 1
Learning rate: 0.1
Stop early: 100 rounds without performance gain on validation data

Reading feature file [training_data.txt]... [Done.]            
(60000 ranked lists, 60000 entries read)
Reading feature file [validation_data.txt]... [Done.]            
(7976 ranked lists, 8434 entries read)
Reading feature file [testing_data.txt]... [Done.]            
(43 ranked lists, 9260 entries read)
Initializing... [Done]
---------------------------------
Training starts...
---------------------------------
#iter   | MAP-T     | MAP-V     | 
---------------------------------
1       | 0.8333    | 0.8749    | 
2       | 0.8333    | 0.8749    | 
3       | 0.8333    | 0.8749    | 
4       | 0.8333    | 0.8749    | 
5       | 0.8333    | 0.8749    | 
6       | 0.8333    | 0.8749    | 
7       | 0.8333    | 0.8749    | 
8       | 0.8333    | 0.8749    | 
9       | 0.8333    | 0.8749    | 
10      | 0.8333    | 0.8749    | 
11      | 0.8333    | 0.8749    | 
12      | 0.8333    | 0.8749    | 
13      | 0.8333    | 0.8749    | 
14      | 0.8333    | 0.8749    | 
15      | 0.8333    | 0.8749    | 
16      | 0.8333    | 0.8749    | 
17      | 0.8333    | 0.8749    | 
18      | 0.8333    | 0.8749    | 
19      | 0.8333    | 0.8749    | 
20      | 0.8333    | 0.8749    | 
21      | 0.8333    | 0.8749    | 
22      | 0.8333    | 0.8749    | 
23      | 0.8333    | 0.8749    | 
24      | 0.8333    | 0.8749    | 
25      | 0.8333    | 0.8749    | 
26      | 0.8333    | 0.8749    | 
27      | 0.8333    | 0.8749    | 
28      | 0.8333    | 0.8749    | 
29      | 0.8333    | 0.8749    | 
30      | 0.8333    | 0.8749    | 
31      | 0.8333    | 0.8749    | 
32      | 0.8333    | 0.8749    | 
33      | 0.8333    | 0.8749    | 
34      | 0.8333    | 0.8749    | 
35      | 0.8333    | 0.8749    | 
36      | 0.8333    | 0.8749    | 
37      | 0.8333    | 0.8749    | 
38      | 0.8333    | 0.8749    | 
39      | 0.8333    | 0.8749    | 
40      | 0.8333    | 0.8749    | 
41      | 0.8333    | 0.8749    | 
42      | 0.8333    | 0.8749    | 
43      | 0.8333    | 0.8749    | 
44      | 0.8333    | 0.8749    | 
45      | 0.8333    | 0.8749    | 
46      | 0.8333    | 0.8749    | 
47      | 0.8333    | 0.8749    | 
48      | 0.8333    | 0.8749    | 
49      | 0.8333    | 0.8749    | 
50      | 0.8333    | 0.8749    | 
51      | 0.8333    | 0.8749    | 
52      | 0.8333    | 0.8749    | 
53      | 0.8333    | 0.8749    | 
54      | 0.8333    | 0.8749    | 
55      | 0.8333    | 0.8749    | 
56      | 0.8333    | 0.8749    | 
57      | 0.8333    | 0.8749    | 
58      | 0.8333    | 0.8749    | 
59      | 0.8333    | 0.8749    | 
60      | 0.8333    | 0.8749    | 
61      | 0.8333    | 0.8749    | 
62      | 0.8333    | 0.8749    | 
63      | 0.8333    | 0.8749    | 
64      | 0.8333    | 0.8749    | 
65      | 0.8333    | 0.8749    | 
66      | 0.8333    | 0.8749    | 
67      | 0.8333    | 0.8749    | 
68      | 0.8333    | 0.8749    | 
69      | 0.8333    | 0.8749    | 
70      | 0.8333    | 0.8749    | 
71      | 0.8333    | 0.8749    | 
72      | 0.8333    | 0.8749    | 
73      | 0.8333    | 0.8749    | 
74      | 0.8333    | 0.8749    | 
75      | 0.8333    | 0.8749    | 
76      | 0.8333    | 0.8749    | 
77      | 0.8333    | 0.8749    | 
78      | 0.8333    | 0.8749    | 
79      | 0.8333    | 0.8749    | 
80      | 0.8333    | 0.8749    | 
81      | 0.8333    | 0.8749    | 
82      | 0.8333    | 0.8749    | 
83      | 0.8333    | 0.8749    | 
84      | 0.8333    | 0.8749    | 
85      | 0.8333    | 0.8749    | 
86      | 0.8333    | 0.8749    | 
87      | 0.8333    | 0.8749    | 
88      | 0.8333    | 0.8749    | 
89      | 0.8333    | 0.8749    | 
90      | 0.8333    | 0.8749    | 
91      | 0.8333    | 0.8749    | 
92      | 0.8333    | 0.8749    | 
93      | 0.8333    | 0.8749    | 
94      | 0.8333    | 0.8749    | 
95      | 0.8333    | 0.8749    | 
96      | 0.8333    | 0.8749    | 
97      | 0.8333    | 0.8749    | 
98      | 0.8333    | 0.8749    | 
99      | 0.8333    | 0.8749    | 
100     | 0.8333    | 0.8749    | 
101     | 0.8333    | 0.8749    | 
102     | 0.8333    | 0.8749    | 
---------------------------------
Finished sucessfully.
MAP on training data: 0.8333
MAP on validation data: 0.8749
---------------------------------
MAP on test data: 0.3987

Model saved to: LambdaMART.txt

```

```
java -jar RankLib-2.15.jar -rank testing_data.txt -load LambdaMART.txt -indri testoutput.trec
```